# Six Degrees Of Wikipedia
**A Project by Robin Graham-Hayes**

In [ ]:
# Run this code to import the nessisary files and libraries
import helpers
import pathways
import wikipedia as wiki

## Introduction

In the Wikipedia game the goal is to get from one Wikipedia page to another in the shortest amount of clicks. You can only use the links in the main article. Talk: pages and other meta Wikipedia pages are un interesting and outside of the heart of the game. 

My project is a deviation of this game. Its goal is to find all the valid pathways between the two pages. It can realy be interesting to find unintuitive pathways between pages. A pathway such as: `| Toothpaste ---> Apricot ---> Alexander the Great |` demostrates how pathways can be unintuitive without the specific context the the articles give. Without this program a lot of these short weird pathways would be hard to find. 

## Methodology

#### The Wikipedia API
The first problem I had to solve was how to interface with wikipedia pages. I could use BeautifulSoup but I was able to find a nice Wikipedia API that made interfacing with wikipedia pretty simple. I used the bash command `pip install wikipedia` to install the API and ran some basic tests:

In [ ]:
mars = wiki.page("Mars (planet)")
mars_links = mars.links
print("Here is a sample of the links:", mars_links[20:25])

Now I have a basic way to get all the links in a page. In addition the Wikipedia API will throw errors if the page does not exist of if it is a disambiguation page which is useful for interacting with broken links and the alternitive format of a disambiguation page.

#### Depth First Searches
The next problem I had to tackle is how to actualy search through wikipedia pages. I knew I needed to specify a maximum depth to search, otherwise the program could end up thousands of pages deep in one pathway it needs to check. I also know that I dont know the precise amount of times I will have to iterate through a search. Following that a recursive depth fisrt search made the most sense for my purpose.

*The Folowing graphic depicts the basic pattern of a depth first search:*

                A  
              / | \  
             B  E  G  
            /\  |   \  
           C  D F    H  

The program starts by searching the first page. It will then go on to serch through the first link, and then through all the following links untill it reaches its max depth in the search. After that it will start going back through the previous branches that were not included in the first deep search. In the above graphic, the search pattern is depicted in alphebetical order.

#### The Algorithm

To create the search helpers `helper.get_pathway()` there are some checks that need to happen first. First we need to check if the file has the end page as a link in the article.

In [ ]:
print(helpers.has_end(mars.title, wiki.page("The Moon")))
print(helpers.has_end(mars.title, wiki.page("Salt")))

Next we need to check if the max depth has been reached, since we are checking the links before hand this check will be one less than the given number.

After those checks we start checking each link in the page recursivly to see if the end page present in that next page. During this step we also check wether the link we are about to search is in the existing trail and if it is we skip it to avoid entering a infinite loop.

#### Slow downs

Once I got the algorithm running, I ran into the problem that accessing thousands and thousands of webpages end up taking a lot of time. To reduce the time I had to limit the maximum depth of my tests to four chains deep. I also rand into the realization the the wikipedia API was pulling all of the links including all of the hundreds of links in the navigation boxes at the bottom of the page. These are typicaly not included in the wikipedia game and it is both uninteresting to look at those paths and significantly slow down the searches.

#### Getting the links



I quickly learned that even with a more robust Wikipedia API there was no way to distinguish links in the navigation boxes from the main links in the article. I slowly developed a way to parse the HTML retrived by the wikipedia API with BeautifulSoup. 
Doing this took a few steps, and is mostly contained in the function `helpers.parse_links()` I had to first seperate the navigation box from the main part of the article. I was able to do this using string indexing, where I found the first index of the navigation box, and cut off all the lines after that index. The next step was to pull only the wikipedia links from the page. This was fairly simple with BeautifulSoup as the search criteria was fairly limited, all the wiki links on the page had a title and no class except disambiguation pages.

BeautifulSoup allows me to use the following functions to find regular links:

`content_soup.find_all(
    name="a", attrs={"class": None, "title": re.compile(".")})`

As well as finding all the disambiguation links:

`content_soup.find_all(
    name="a", attrs={"class": "mw-disambig", "title": re.compile(".")})`
    
We then only have to sort out all the meta pages. The function `helpers.get_titles()` which takes the ResultsSet object from `helpers.parse_links()` and only returns the titles that are from articles that are regular wikipedia pages.

All of this was then simplified into the function `helpers.get_links()` which parses the links of any page title given to it.

In [ ]:
print(f"The page {mars.title} has {len(mars_links)} links")
print(f"The page {mars.title} has {len(helpers.get_links(mars.title))} links")

As you can see above this significantly reduces the number of links the program has to sort through.

#### Saving to files

To further save time I decided to save information localy. I created the function `helpers.save_links()` to save all the links to a file witht the title of the article and each link of the page listed on seperate lines of the file. The function `helpers.read_links()` was created to read the links from the file. Both of these functions were added to `helpers.get_links()` and significantly sped up the runtime of the search.


#### Making an Interactive Script


The result of all of these functions was `pathways.find_paths()` however I wanted to provide a way for any user to interact with the program and not need to know the ins and outs of how `pathways.find_paths()` works. I created the python script `Six_Degrees_of_Wiki.py` to do just that.

By running `python Six_Degrees_of_Wiki.py` in a bash terminal you will be prompted for the strarting article and if you would like to use the default arguments for the max steps and ending page. If you don't it will prompt you for those as well.

## Results

The results collected from the main function call are filterd through `helpers.plot_all_paths()` to format them in a more visualy and are then saved to a file. 

For example the following function call produces a file containing the following text:

We can call `pathways.find_paths` to see the full out put of the function call:

In [ ]:
# The following code will produce ~125 lines of text
# to show you the process the search follows
pathways.find_paths("Toothpaste", "Alexander the Great", 2)

If you use `python Six_Degrees_of_Wiki.py` to run the same search as the function call above does you would get the following terminal output:

## Conclusion

I learned a lot through this project, I got alot more used to using an API to request data. I also learned a lot about BeautifulSoup, though I still have a lot to learn about using BeautifulSoup. Also learning the limits of programs and how a large program will actualy take time to run, as well as learning how to optimise a program to increase runtime efficiency. If you can avoid a try-except call without risking a error being thrown, that can be extreamly useful as they tend to add a lot of time when call repeatedly.

I think there are some really interesting paths between wikipedia pages that are for the most part unintuitive. Who knew you could get from Ketchup to The Bible with the path: `| Ketchup --> Salt --> Bible |` or from Toothpaste to Alexander the Great through apricot: `| Toothpaste ---> Apricot ---> Alexander the Great |`

I think this project highlights how important context is. If you are just told something or given some information without context how would you know what to do with it. If I was playing the wikipedia game I would never click on Apricot to get to Alexander the Great. Things are not ment to existt in this world without any context. A student is not defined by their SAT scores, you need to consider the bigger picture.